### MNIST

In [1]:

from tensorflow.keras.datasets import mnist, fashion_mnist, cifar10
from tensorflow.keras.utils import to_categorical

(x_train, y_train), (x_test, y_test) = mnist.load_data()
num_classes = y_train.max()-y_train.min()+1
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)
x_train.shape

2025-05-12 19:39:27.423069: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747067967.525346   80307 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747067967.555504   80307 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747067967.771051   80307 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747067967.771098   80307 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747067967.771101   80307 computation_placer.cc:177] computation placer alr

(60000, 28, 28)

### NN 1

In [2]:
num_pixels = x_test.shape[1]*x_test.shape[2]
x_train_resh = x_train.reshape((x_train.shape[0], num_pixels))
x_test_resh = x_test.reshape((x_test.shape[0], num_pixels))
x_train_resh.shape


(60000, 784)

In [3]:
from tensorflow.keras.layers import Dense
from tensorflow.keras import Input, Model

inputs = Input(shape=(x_train_resh.shape[1],))
x = Dense(512, activation="relu")(inputs)
x = Dense(256, activation="relu")(x)
x = Dense(10, activation="softmax")(x)
model0 = Model(inputs=inputs, outputs=x)
model0.summary()

2025-05-12 19:39:35.278568: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       401,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 535,818 (2.04 MB)

 Trainable params: 535,818 (2.04 MB)

 Non-trainable params: 0 (0.00 B)

In [4]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

model0.compile(optimizer='adam',loss='categorical_crossentropy',
              metrics=['accuracy','precision','recall'])
model0.fit(x_train_resh, y_train, epochs=20, batch_size=128, validation_split=0.25, verbose=1)
loss, acc, prec, rec = model0.evaluate(x_test_resh,y_test)
print(f'accuracy: {acc},\n precision: {prec},\n recall: {rec}')

pred = model0.predict(x_test_resh)
y_pred = np.argmax(pred, axis=1)
y_test0 = np.argmax(y_test, axis=1)
print(confusion_matrix(y_test0,y_pred))
print(classification_report(y_test0, y_pred))


Epoch 1/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 9s 20ms/step - accuracy: 0.8664 - loss: 0.4717 - precision: 0.9306 - recall: 0.7859 - val_accuracy: 0.9605 - val_loss: 0.1335 - val_precision: 0.9680 - val_recall: 0.9545
Epoch 2/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.9714 - loss: 0.0947 - precision: 0.9760 - recall: 0.9667 - val_accuracy: 0.9565 - val_loss: 0.1392 - val_precision: 0.9611 - val_recall: 0.9523
Epoch 3/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.9806 - loss: 0.0605 - precision: 0.9836 - recall: 0.9783 - val_accuracy: 0.9717 - val_loss: 0.0936 - val_precision: 0.9744 - val_recall: 0.9700
Epoch 4/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - accuracy: 0.9893 - loss: 0.0357 - precision: 0.9905 - recall: 0.9877 - val_accuracy: 0.9756 - val_loss: 0.0872 - val_precision: 0.9781 - val_recall: 0.9734
Epoch 5/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - accuracy: 0.9912 - loss: 0.0276 - precision: 0.9923 - recall: 0.9904 - val_accuracy: 0.9719 - val_los

### CNN 1

In [5]:

from tensorflow.keras.layers import Conv2D,MaxPooling2D, Flatten

inputs = Input(shape=(28, 28, 1))
x = Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
x = MaxPooling2D(pool_size=2)(x)
x = Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = MaxPooling2D(pool_size=2)(x)
x = Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = Flatten()(x)
outputs = Dense(10, activation="softmax")(x)
model = Model(inputs=inputs, outputs=outputs)
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 28, 28, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 3, 3, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1152)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │        11,530 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 104,202 (407.04 KB)

 Trainable params: 104,202 (407.04 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:

model.compile(optimizer='adam',loss='categorical_crossentropy',
              metrics=['accuracy','precision','recall'])
model.fit(x_train, y_train, epochs=10, batch_size=128, validation_split=0.25, verbose=1)
loss, acc, prec, rec = model.evaluate(x_test,y_test)
print(f'accuracy: {acc},\n precision: {prec},\n recall: {rec}')




Epoch 1/10
352/352 ━━━━━━━━━━━━━━━━━━━━ 25s 65ms/step - accuracy: 0.8206 - loss: 0.6383 - precision: 0.9032 - recall: 0.7022 - val_accuracy: 0.9753 - val_loss: 0.0868 - val_precision: 0.9794 - val_recall: 0.9709
Epoch 2/10
352/352 ━━━━━━━━━━━━━━━━━━━━ 20s 58ms/step - accuracy: 0.9784 - loss: 0.0733 - precision: 0.9820 - recall: 0.9741 - val_accuracy: 0.9819 - val_loss: 0.0620 - val_precision: 0.9846 - val_recall: 0.9803
Epoch 3/10
352/352 ━━━━━━━━━━━━━━━━━━━━ 22s 62ms/step - accuracy: 0.9864 - loss: 0.0448 - precision: 0.9879 - recall: 0.9849 - val_accuracy: 0.9825 - val_loss: 0.0553 - val_precision: 0.9842 - val_recall: 0.9814
Epoch 4/10
352/352 ━━━━━━━━━━━━━━━━━━━━ 21s 59ms/step - accuracy: 0.9883 - loss: 0.0362 - precision: 0.9895 - recall: 0.9876 - val_accuracy: 0.9787 - val_loss: 0.0649 - val_precision: 0.9810 - val_recall: 0.9770
Epoch 5/10
352/352 ━━━━━━━━━━━━━━━━━━━━ 22s 63ms/step - accuracy: 0.9897 - loss: 0.0309 - precision: 0.9909 - recall: 0.9889 - val_accuracy: 0.9856 - va

In [7]:
pred = model.predict(x_test)
y_pred = np.argmax(pred, axis=1)
y_test = np.argmax(y_test, axis=1)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test, y_pred))

313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step
[[ 973    0    2    0    0    0    2    2    1    0]
 [   0 1124    1    3    0    1    2    3    0    1]
 [   0    0 1027    1    0    0    0    3    1    0]
 [   0    0    1 1005    0    3    0    0    1    0]
 [   0    0    1    0  967    0    4    0    0   10]
 [   0    0    0    4    0  887    1    0    0    0]
 [   3    2    2    0    1    4  944    0    2    0]
 [   0    1    3    1    0    0    0 1018    1    4]
 [   2    0    2    4    1    3    0    3  950    9]
 [   2    0    0    0    1    3    0    1    0 1002]]
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       980
           1       1.00      0.99      0.99      1135
           2       0.99      1.00      0.99      1032
           3       0.99      1.00      0.99      1010
           4       1.00      0.98      0.99       982
           5       0.98      0.99      0.99       892
           6       0.99      0.99      0.99      

## FASHION MNIST

In [8]:
(x_train2, y_train2), (x_test2, y_test2) = fashion_mnist.load_data()
num_classes = y_train2.max()-y_train2.min()+1
x_train2 = x_train2.astype("float32") / 255
x_test2 = x_test2.astype("float32") / 255
y_train2 = to_categorical(y_train2, num_classes)
y_test2 = to_categorical(y_test2, num_classes)

### NN 2

In [9]:
num_pixels = x_test2.shape[1]*x_test2.shape[2]
x_train2_resh = x_train2.reshape((x_train2.shape[0], num_pixels))
x_test2_resh = x_test2.reshape((x_test2.shape[0], num_pixels))
x_train2_resh.shape

(60000, 784)

In [10]:
inputs = Input(shape=(x_train2_resh.shape[1],)) 
x = Dense(1024, activation="relu")(inputs)
x = Dense(512, activation="relu")(x)
x = Dense(256, activation="relu")(x)
x = Dense(10, activation="softmax")(x)
model0_2 = Model(inputs=inputs, outputs=x)
model0_2.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1024)           │       803,840 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,462,538 (5.58 MB)

 Trainable params: 1,462,538 (5.58 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
model0_2.compile(loss="categorical_crossentropy", optimizer="adam", 
                 metrics=['accuracy','precision','recall'])
model0_2.fit(x_train2_resh, y_train2, epochs=70, batch_size=128, validation_split=0.25, verbose=1)
loss, acc, prec, rec = model0_2.evaluate(x_test2_resh,y_test2)
print(f'accuracy: {acc},\n precision: {prec},\n recall: {rec}')


Epoch 1/70
352/352 ━━━━━━━━━━━━━━━━━━━━ 15s 35ms/step - accuracy: 0.7636 - loss: 0.6647 - precision: 0.8469 - recall: 0.6854 - val_accuracy: 0.8408 - val_loss: 0.4222 - val_precision: 0.8721 - val_recall: 0.8107
Epoch 2/70
352/352 ━━━━━━━━━━━━━━━━━━━━ 12s 35ms/step - accuracy: 0.8638 - loss: 0.3714 - precision: 0.8931 - recall: 0.8336 - val_accuracy: 0.8658 - val_loss: 0.3623 - val_precision: 0.8947 - val_recall: 0.8398
Epoch 3/70
352/352 ━━━━━━━━━━━━━━━━━━━━ 14s 40ms/step - accuracy: 0.8795 - loss: 0.3251 - precision: 0.9039 - recall: 0.8570 - val_accuracy: 0.8757 - val_loss: 0.3401 - val_precision: 0.9011 - val_recall: 0.8507
Epoch 4/70
352/352 ━━━━━━━━━━━━━━━━━━━━ 12s 33ms/step - accuracy: 0.8897 - loss: 0.3012 - precision: 0.9101 - recall: 0.8698 - val_accuracy: 0.8807 - val_loss: 0.3237 - val_precision: 0.8978 - val_recall: 0.8651
Epoch 5/70
352/352 ━━━━━━━━━━━━━━━━━━━━ 13s 36ms/step - accuracy: 0.8928 - loss: 0.2846 - precision: 0.9122 - recall: 0.8747 - val_accuracy: 0.8741 - va

In [12]:
pred2 = model0_2.predict(x_test2_resh)
y_pred2 = np.argmax(pred2, axis=1)
y_test02 = np.argmax(y_test2, axis=1)
print(confusion_matrix(y_test02,y_pred2))
print(classification_report(y_test02, y_pred2))

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step
[[842   1  16  17   3   1 112   0   8   0]
 [  1 980   2   7   4   0   5   0   1   0]
 [ 26   0 843   8  59   0  64   0   0   0]
 [ 21   5  15 907  26   0  22   0   4   0]
 [  0   1 111  35 765   0  87   0   1   0]
 [  0   0   0   0   0 970   0  21   0   9]
 [115   0  76  35  44   0 720   0  10   0]
 [  0   0   0   0   0   6   0 957   0  37]
 [  3   0   5   5   2   1   4   5 975   0]
 [  0   0   0   0   0   7   1  30   0 962]]
              precision    recall  f1-score   support

           0       0.84      0.84      0.84      1000
           1       0.99      0.98      0.99      1000
           2       0.79      0.84      0.82      1000
           3       0.89      0.91      0.90      1000
           4       0.85      0.77      0.80      1000
           5       0.98      0.97      0.98      1000
           6       0.71      0.72      0.71      1000
           7       0.94      0.96      0.95      1000
           8       0.98      0.97      0.

### CNN 2

In [13]:
inputs = Input(shape=(28, 28, 1))
x = Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
x = MaxPooling2D(pool_size=2)(x)
x = Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = MaxPooling2D(pool_size=2)(x)
x = Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = Flatten()(x)
outputs = Dense(10, activation="softmax")(x)
model2 = Model(inputs=inputs, outputs=outputs)
model2.summary()


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 28, 28, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 3, 3, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 1152)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 10)             │        11,530 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 104,202 (407.04 KB)

 Trainable params: 104,202 (407.04 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
model2 = Model(inputs=inputs, outputs=outputs)
model2.compile(optimizer='adam',loss='categorical_crossentropy',
              metrics=['accuracy','precision','recall'])

model2.fit(x_train2, y_train2, epochs=20, batch_size=128, validation_split=0.25, verbose=1)
loss, acc, prec, rec = model2.evaluate(x_test2,y_test2)
print(f'accuracy: {acc},\n precision: {prec},\n recall: {rec}')

Epoch 1/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 29s 74ms/step - accuracy: 0.6724 - loss: 0.9457 - precision: 0.8136 - recall: 0.5132 - val_accuracy: 0.8349 - val_loss: 0.4476 - val_precision: 0.8782 - val_recall: 0.7899
Epoch 2/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 25s 72ms/step - accuracy: 0.8462 - loss: 0.4201 - precision: 0.8809 - recall: 0.8093 - val_accuracy: 0.8630 - val_loss: 0.3779 - val_precision: 0.8866 - val_recall: 0.8417
Epoch 3/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 24s 68ms/step - accuracy: 0.8768 - loss: 0.3498 - precision: 0.9011 - recall: 0.8529 - val_accuracy: 0.8805 - val_loss: 0.3319 - val_precision: 0.9024 - val_recall: 0.8601
Epoch 4/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 24s 67ms/step - accuracy: 0.8871 - loss: 0.3090 - precision: 0.9078 - recall: 0.8693 - val_accuracy: 0.8840 - val_loss: 0.3157 - val_precision: 0.9027 - val_recall: 0.8665
Epoch 5/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 23s 65ms/step - accuracy: 0.8964 - loss: 0.2855 - precision: 0.9138 - recall: 0.8795 - val_accuracy: 0.8915 - va

In [15]:
pred2 = model2.predict(x_test2)
y_pred2 = np.argmax(pred2, axis=1)
y_test2 = np.argmax(y_test2, axis=1)
print(confusion_matrix(y_test2,y_pred2))
print(classification_report(y_test2, y_pred2))

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step
[[840   0  33  14   1   1 103   0   8   0]
 [  1 983   2   9   2   0   1   0   2   0]
 [ 10   1 889   7  43   0  48   0   2   0]
 [ 21   2  17 901  30   0  27   0   2   0]
 [  2   1  51  25 864   1  53   0   3   0]
 [  0   0   0   0   0 989   0   4   0   7]
 [105   2  74  17  75   0 720   0   7   0]
 [  0   0   0   0   0  47   0 941   1  11]
 [  3   0   5   1   3   1   7   3 977   0]
 [  0   0   0   0   0   5   1  43   0 951]]
              precision    recall  f1-score   support

           0       0.86      0.84      0.85      1000
           1       0.99      0.98      0.99      1000
           2       0.83      0.89      0.86      1000
           3       0.93      0.90      0.91      1000
           4       0.85      0.86      0.86      1000
           5       0.95      0.99      0.97      1000
           6       0.75      0.72      0.73      1000
           7       0.95      0.94      0.95      1000
           8       0.98      0.98      0.

## CIFAR 10

In [16]:
(x_train3, y_train3), (x_test3, y_test3) = cifar10.load_data()
num_classes = y_train3.max()-y_train3.min()+1
x_train3 = x_train3.astype("float32") / 255
x_test3 = x_test3.astype("float32") / 255
y_train3 = to_categorical(y_train3, num_classes)
y_test3 = to_categorical(y_test3, num_classes)

### NN 3

In [21]:
inputs = Input(shape=x_train3.shape[1:])
x = Flatten()(inputs)
x = Dense(1024, activation="relu")(x)
x = Dense(512, activation="relu")(x)
x = Dense(512, activation="relu")(x)
x = Dense(256, activation="relu")(x)
outputs = Dense(10, activation="softmax")(x)
model0_3 = Model(inputs=inputs, outputs=outputs)

In [22]:
model0_3.compile(loss="categorical_crossentropy", optimizer="adam", 
               metrics=['accuracy','precision','recall'])
model0_3.fit(x_train3, y_train3, epochs=100, batch_size=128, validation_split=0.25, verbose=1)
loss, acc, prec, rec = model0_3.evaluate(x_test3,y_test3)
print(f'accuracy: {acc},\n precision: {prec},\n recall: {rec}')

Epoch 1/100
293/293 ━━━━━━━━━━━━━━━━━━━━ 27s 82ms/step - accuracy: 0.2248 - loss: 2.1348 - precision: 0.4399 - recall: 0.0222 - val_accuracy: 0.3530 - val_loss: 1.8018 - val_precision: 0.5650 - val_recall: 0.1206
Epoch 2/100
293/293 ━━━━━━━━━━━━━━━━━━━━ 22s 74ms/step - accuracy: 0.3628 - loss: 1.7566 - precision: 0.6088 - recall: 0.1153 - val_accuracy: 0.3927 - val_loss: 1.7165 - val_precision: 0.5816 - val_recall: 0.1842
Epoch 3/100
293/293 ━━━━━━━━━━━━━━━━━━━━ 22s 74ms/step - accuracy: 0.4039 - loss: 1.6526 - precision: 0.6319 - recall: 0.1605 - val_accuracy: 0.4198 - val_loss: 1.6262 - val_precision: 0.6780 - val_recall: 0.1521
Epoch 4/100
293/293 ━━━━━━━━━━━━━━━━━━━━ 21s 72ms/step - accuracy: 0.4341 - loss: 1.5748 - precision: 0.6532 - recall: 0.1923 - val_accuracy: 0.4292 - val_loss: 1.5839 - val_precision: 0.6162 - val_recall: 0.2323
Epoch 5/100
293/293 ━━━━━━━━━━━━━━━━━━━━ 20s 69ms/step - accuracy: 0.4571 - loss: 1.5158 - precision: 0.6718 - recall: 0.2318 - val_accuracy: 0.4352

In [23]:
pred3 = model0_3.predict(x_test3)
y_pred3 = np.argmax(pred3, axis=1)
y_test03 = np.argmax(y_test3, axis=1)
print(confusion_matrix(y_test03,y_pred3))
print(classification_report(y_test03, y_pred3))

313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step
[[595  27  43  46  35  31  21  32 113  57]
 [ 55 540  15  19  22  27  22  29  65 206]
 [ 86  21 363 111 127 100  94  56  20  22]
 [ 37  21 109 317  77 173 135  48  33  50]
 [ 63  10 179  75 364  67 134  69  22  17]
 [ 37   9 101 225  92 347  85  54  24  26]
 [ 15  12 121  99 113  49 547  19   6  19]
 [ 61  14  71  74  84 101  31 497  20  47]
 [117  71  24  33  30  16  14  18 610  67]
 [ 67 159  22  34  25  28  30  36  77 522]]
              precision    recall  f1-score   support

           0       0.53      0.59      0.56      1000
           1       0.61      0.54      0.57      1000
           2       0.35      0.36      0.35      1000
           3       0.31      0.32      0.31      1000
           4       0.38      0.36      0.37      1000
           5       0.37      0.35      0.36      1000
           6       0.49      0.55      0.52      1000
           7       0.58      0.50      0.53      1000
           8       0.62      0.61      0

### CNN 3

In [25]:
inputs = Input(shape=x_train3.shape[1:])
x = Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
x = MaxPooling2D(pool_size=2)(x)
x = Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = MaxPooling2D(pool_size=2)(x)
x = Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = Flatten()(x)
outputs = Dense(10, activation="softmax")(x)
model3 = Model(inputs=inputs, outputs=outputs)
model3.compile(optimizer='adam',loss='categorical_crossentropy',
              metrics=['accuracy','precision','recall'])

model3.fit(x_train3, y_train3, epochs=20, batch_size=128, validation_split=0.25, verbose=1)
loss, acc, prec, rec = model3.evaluate(x_test3,y_test3)
print(f'accuracy: {acc},\n precision: {prec},\n recall: {rec}')

Epoch 1/20
293/293 ━━━━━━━━━━━━━━━━━━━━ 36s 111ms/step - accuracy: 0.3030 - loss: 1.9025 - precision: 0.5587 - recall: 0.0530 - val_accuracy: 0.4840 - val_loss: 1.4455 - val_precision: 0.7061 - val_recall: 0.2089
Epoch 2/20
293/293 ━━━━━━━━━━━━━━━━━━━━ 26s 88ms/step - accuracy: 0.5033 - loss: 1.3864 - precision: 0.7138 - recall: 0.2711 - val_accuracy: 0.5490 - val_loss: 1.2856 - val_precision: 0.7219 - val_recall: 0.3387
Epoch 3/20
293/293 ━━━━━━━━━━━━━━━━━━━━ 25s 87ms/step - accuracy: 0.5666 - loss: 1.2102 - precision: 0.7423 - recall: 0.3785 - val_accuracy: 0.5756 - val_loss: 1.2008 - val_precision: 0.7155 - val_recall: 0.4106
Epoch 4/20
293/293 ━━━━━━━━━━━━━━━━━━━━ 26s 89ms/step - accuracy: 0.6153 - loss: 1.1022 - precision: 0.7635 - recall: 0.4492 - val_accuracy: 0.6202 - val_loss: 1.0918 - val_precision: 0.7580 - val_recall: 0.4626
Epoch 5/20
293/293 ━━━━━━━━━━━━━━━━━━━━ 26s 90ms/step - accuracy: 0.6515 - loss: 1.0006 - precision: 0.7858 - recall: 0.5090 - val_accuracy: 0.6354 - v

In [26]:
pred3 = model3.predict(x_test3)
y_pred3 = np.argmax(pred3, axis=1)
y_test3 = np.argmax(y_test3, axis=1)
print(confusion_matrix(y_test3,y_pred3))
print(classification_report(y_test3, y_pred3))

313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step
[[794  25  27   9  11   7  13   6  73  35]
 [ 27 838   2   8   1   7  11   4  30  72]
 [ 94  19 534  65  81  78  68  34  14  13]
 [ 34  17  57 496  68 186  64  28  26  24]
 [ 35  11  66  64 631  41  56  67  15  14]
 [ 17   5  45 154  40 642  26  48  15   8]
 [ 10   7  37  56  25  25 818  11   8   3]
 [ 35   5  15  43  73  71  10 716   6  26]
 [ 66  43   5   9   1   7   3   5 833  28]
 [ 31 116   4  12   6   7   7  11  35 771]]
              precision    recall  f1-score   support

           0       0.69      0.79      0.74      1000
           1       0.77      0.84      0.80      1000
           2       0.67      0.53      0.60      1000
           3       0.54      0.50      0.52      1000
           4       0.67      0.63      0.65      1000
           5       0.60      0.64      0.62      1000
           6       0.76      0.82      0.79      1000
           7       0.77      0.72      0.74      1000
           8       0.79      0.83      0